In [1]:
from __future__ import print_function

from sklearn.preprocessing import OneHotEncoder
from keras.layers.core import Dense, Activation, Dropout
from keras.preprocessing import sequence
from keras.models import Model
from keras.layers import Dense, Embedding, Bidirectional, Input, TimeDistributed
from keras.layers import LSTM
from keras.datasets import imdb
from keras import optimizers
import pandas as pd
import numpy as np
import os

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#parameters
maxlen = 30
labels = 2

In [3]:
czech = pd.read_excel('czech.xlsx', encoding='latin',header = None)
czech.columns = ['Name', 'm_or_f']
czech['namelen'] = [len(str(i)) for i in czech['Name']]


In [4]:
cname = czech['Name']
collect = []


for i in range(len(cname)):
    collect.extend(list(str(cname[i]).lower()))
# collect.extend(['END'])
collect = set(collect)

czech.head()

,Name,m_or_f,namelen
0,Abigail,f,7
1,Ada,f,3
2,Adalberta,f,9
3,Adéla,f,5
4,Adelaida,f,8


In [5]:
data_set = pd.read_csv("gender_data.csv",header=None)
data_set.columns = ['name','m_or_f']
data_set['namelen']= [len(str(i)) for i in data_set['name']]
data_set1 = data_set[(data_set['namelen'] >= 2) ]

In [6]:
data_set1.groupby('m_or_f')['name'].count()

m_or_f
f    6705
m    8475
Name: name, dtype: int64

In [7]:
names = data_set['name']
gender = data_set['m_or_f']
vocab = set(' '.join([str(i) for i in names]))
vocab.add('END')
vocab = vocab.union(collect)
len_vocab = len(vocab)

In [8]:
print(vocab)
print("vocab length is ",len_vocab)
print ("length of data_set is ",len(data_set1))

{'p', 'n', 'END', 'a', 'í', 's', 'ď', '0', 't', '2', 'd', 'ů', 'ó', 'u', 'i', 'y', 'ň', '.', 'č', 'b', '9', 'c', 'ž', 'k', 'h', 'á', 'ř', '8', 'f', 'm', '5', '3', 'ť', 'e', 'o', 'š', 'ě', '4', 'r', 'ú', '7', 'é', 'z', 'x', 'q', 'g', '1', ' ', 'v', 'l', 'j', 'w', '6', 'ý'}
vocab length is  54
length of data_set is  15226


In [9]:
char_index = dict((c, i) for i, c in enumerate(vocab))

In [10]:
print(char_index)

{'p': 0, 'n': 1, 'END': 2, 'a': 3, 's': 5, 'ď': 6, '0': 7, 't': 8, '2': 9, 'd': 10, 'ů': 11, 'ó': 12, 'u': 13, 'i': 14, 'y': 15, 'ň': 16, 'í': 4, '.': 17, 'b': 19, '9': 20, '5': 30, 'c': 21, 'g': 45, 'ž': 22, 'k': 23, 'h': 24, 'á': 25, 'ř': 26, '8': 27, 'f': 28, 'm': 29, '3': 31, 'ť': 32, 'e': 33, 'o': 34, 'š': 35, 'ě': 36, '4': 37, 'ú': 39, '7': 40, 'é': 41, 'z': 42, 'x': 43, 'q': 44, 'č': 18, '1': 46, 'r': 38, ' ': 47, 'v': 48, 'l': 49, 'j': 50, 'w': 51, '6': 52, 'ý': 53}


In [11]:
#train test split
msk = np.random.rand(len(data_set1)) < 0.8
train = data_set1[msk]
test = data_set1[~msk]     


In [12]:
def set_flag(i):
    tmp = np.zeros(len_vocab);
    tmp[i] = 1
    return(tmp)

In [13]:
set_flag(3)

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

#### modify the code above to also convert each index to one-hot encoded representation

In [14]:
#take data_set upto max and truncate rest
#encode to vector space(one hot encoding)
#padd 'END' to shorter sequences
#also convert each index to one-hot encoding
train_X = []
train_Y = []
trunc_train_name = [str(i)[0:maxlen] for i in train.name]
for i in trunc_train_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    train_X.append(tmp)
for i in train.m_or_f:
    if i == 'm':
        train_Y.append([1,0])
    else:
        train_Y.append([0,1])
    
train_X=np.asarray(train_X)
train_Y=np.asarray(train_Y)

In [15]:
test_X = []
test_Y = []
trunc_test_name = [str(i)[0:maxlen] for i in test.name]
for i in trunc_test_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    test_X.append(tmp)
for i in test.m_or_f:
    if i == 'm':
        test_Y.append([1,0])
    else:
        test_Y.append([0,1])
    
test_X = np.asarray(test_X)
test_Y = np.asarray(test_Y)

In [16]:
print(np.asarray(test_X).shape)
print(np.asarray(test_Y).shape)

(3045, 30, 54)
(3045, 2)


In [17]:
msk = np.random.rand(len(czech)) < 0.8


vtrain = czech[msk]
vtest = czech[~msk]

In [18]:
vtrain_x = []
vtrain_y = []

train_name = [str(i) for i in vtrain.Name]
for i in train_name:
    tmp = [set_flag(char_index[j]) for j in str(i.lower())]
    for k in range(0, maxlen - len(str(i))):
        tmp.append(set_flag(char_index['END']))
    vtrain_x.append(tmp)
for i in vtrain.m_or_f:
    if i == 'm':
        vtrain_y.append([1,0])
    else:
        vtrain_y.append([0,1])
vtrain_x = np.asarray(vtrain_x)
vtrain_y = np.asarray(vtrain_y)

In [19]:
vtest_x = []
vtest_y = []

train_name = [str(i) for i in vtest.Name]
for i in train_name:
    tmp = [set_flag(char_index[j]) for j in str(i.lower())]
    for k in range(0, maxlen - len(str(i))):
        tmp.append(set_flag(char_index['END']))
    vtest_x.append(tmp)
for i in vtest.m_or_f:
    if i == 'm':
        vtest_y.append([1,0])
    else:
        vtest_y.append([0,1])
vtest_x = np.asarray(vtest_x)
vtest_y = np.asarray(vtest_y)

#### build model in keras ( a stacked LSTM model with many-to-one arch ) here 30 sequence and 2 output each for one category(m/f)

In [20]:
#build the model: 2 stacked LSTM
print('Build model...')
input_bilstm=Input(shape = (maxlen,len_vocab))
bi_one = Bidirectional(LSTM(512, return_sequences=True))(input_bilstm)
drop1 = Dropout(0.2)(bi_one)
bi_two = Bidirectional(LSTM(512, return_sequences=False))(drop1)
drop2 = Dropout(0.2)(bi_two)
output = Dense(2, activation='softmax')(drop2)
model = Model(input_bilstm, output)


optimizer = optimizers.adam(lr = 0.01)
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])




Build model...


In [25]:
batch_size=500
model.fit(train_X, train_Y,
          batch_size=batch_size,
          epochs=50,
          validation_data=(vtrain_x, vtrain_y)
         )
model.save('Martin_program.h5')

Train on 12181 samples, validate on 1031 samples
Epoch 1/50
12181/12181 [==============================] - 13s 1ms/step - loss: 0.3253 - acc: 0.8671 - val_loss: 0.3262 - val_acc: 0.9040
Epoch 2/50
12181/12181 [==============================] - 13s 1ms/step - loss: 0.3124 - acc: 0.8728 - val_loss: 0.3282 - val_acc: 0.8914
Epoch 3/50
12181/12181 [==============================] - 13s 1ms/step - loss: 0.3076 - acc: 0.8723 - val_loss: 0.3248 - val_acc: 0.8991
Epoch 4/50
12181/12181 [==============================] - 13s 1ms/step - loss: 0.2972 - acc: 0.8811 - val_loss: 0.3115 - val_acc: 0.9020
Epoch 5/50
12181/12181 [==============================] - 13s 1ms/step - loss: 0.2834 - acc: 0.8880 - val_loss: 0.3155 - val_acc: 0.8982
Epoch 6/50
12181/12181 [==============================] - 13s 1ms/step - loss: 0.2759 - acc: 0.8914 - val_loss: 0.3359 - val_acc: 0.8923
Epoch 7/50
12181/12181 [==============================] - 13s 1ms/step - loss: 0.2725 - acc: 0.8934 - val_loss: 0.3418 - val_acc:

In [24]:
score, acc = model.evaluate(vtest_x, vtest_y)
print('Test score:', score)
print('Test accuracy:', acc)

219/219 [==============================] - 0s 885us/step
Test score: 0.3811889509223912
Test accuracy: 0.8493150665879794


In [148]:
name=["riya"]
X=[]
trunc_name = [i[0:maxlen] for i in name]
for i in trunc_name:
    tmp = [set_flag(char_index[j]) for j in str(i.lower())]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    X.append(tmp)
pred=model.predict(np.asarray(X))
pred

array([[0.16779284, 0.83220714]], dtype=float32)

In [47]:
pred

array([[0.05657073, 0.9434293 ],
       [0.9473468 , 0.05265319],
       [0.9572235 , 0.04277651]], dtype=float32)

#### Lets train more, clearly some very simple female names it doesnt get right like mentioned above (inspite it exists in training data)

In [29]:
score, acc = model.evaluate(vtest_x, vtest_y)
print('Test score:', score)
print('Test accuracy:', acc)

219/219 [==============================] - 0s 980us/step
Test score: 0.6552725690140572
Test accuracy: 0.817351598990018


In [49]:
name=["sandhya","jaspreet","rajesh","kaveri","aditi deepak","arihant","sasikala","aditi","ragini rajaram"]
X=[]
trunc_name = [i[0:maxlen] for i in name]
for i in trunc_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    X.append(tmp)
pred=model.predict(np.asarray(X))
pred

array([[5.6570753e-02, 9.4342929e-01],
       [9.4734681e-01, 5.2653167e-02],
       [9.5722347e-01, 4.2776532e-02],
       [3.5088859e-02, 9.6491110e-01],
       [2.1997560e-04, 9.9978000e-01],
       [9.9858874e-01, 1.4111887e-03],
       [1.1531739e-04, 9.9988472e-01],
       [4.6136803e-03, 9.9538630e-01],
       [1.8581923e-02, 9.8141807e-01]], dtype=float32)

In [33]:
name=["abhi","abhi deepak","mr. abhi"]
X=[]
trunc_name = [i[0:maxlen] for i in name]
for i in trunc_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    X.append(tmp)
pred=model.predict(np.asarray(X))
pred

array([[0.4178515 , 0.5821485 ],
       [0.37405205, 0.62594795],
       [0.02188767, 0.9781123 ]], dtype=float32)

In [34]:
name=["rajini","rajinikanth","mr. rajini"]
X=[]
trunc_name = [i[0:maxlen] for i in name]
for i in trunc_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    X.append(tmp)
pred=model.predict(np.asarray(X))
pred

array([[0.09026915, 0.90973085],
       [0.9785337 , 0.02146639],
       [0.03325909, 0.9667409 ]], dtype=float32)

In [35]:
#save our model and data
model.save_weights('gender_model',overwrite=True)
train.to_csv("train_split.csv")
test.to_csv("test_split.csv")

In [36]:
evals = model.predict(test_X)
prob_m = [i[0] for i in evals]

In [37]:
out = pd.DataFrame(prob_m)
out['name'] = test.name.reset_index()['name']
out['m_or_f']=test.m_or_f.reset_index()['m_or_f']

In [38]:
out.head(10)
out.columns = ['prob_m','name','actual']
out.head(10)
out.to_csv("gender_pred_out.csv")

In [150]:
check = pd.read_csv('name_gender.csv')

In [151]:
check.shape

(95026, 3)

In [152]:
check.head()


,name,gender,probability
0,Aaban,M,1.0
1,Aabha,F,1.0
2,Aabid,M,1.0
3,Aabriella,F,1.0
4,Aada,F,1.0


In [ ]:
check = check[]